In [1]:
import requests
import pandas as pd
from datetime import date
from io import BytesIO

from datetime import datetime
from dateutil.parser import parse
from tqdm import tqdm

import pymongo
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [2]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

In [3]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("KMTC")

In [4]:
# array for failed tracking
gagal=[]

In [5]:
hasil_akhir = []
for i, bls in enumerate(tqdm(bl_list)):
    try:
        # hitting the API
        url_ctr = "https://api.ekmtc.com/trans/trans/cargo-tracking/{}?dtKnd=BL&blNo={}".format(
            bls[4:], bls[4:])
        response_ctr = requests.request("GET", url_ctr)

        # getting container data. booking id needed for API URL
        ctr_data = response_ctr.json()
        list_of_container_number = []
        list_of_bookingid = []

        for j, ctr_info in enumerate(ctr_data['cntrList']):
            list_of_container_number.append(ctr_info["cntrNo"])
            list_of_bookingid.append(ctr_info["bkgNo"])

        for c, ctr in enumerate(list_of_container_number):
            url_milestone = "https://api.ekmtc.com/trans/trans/cargo-tracking/{}/detail?bkgNo={}&cntrNo={}&dtKnd=BL&strBkgNo={}".format(
                bls[4:], list_of_bookingid[c], ctr, list_of_bookingid[c])
            
            # hitting the API again for every container for container milestones
            response_milestone = requests.request("GET", url_milestone)

            # aqcuiring milestone datas for every container
            milstone_data = response_milestone.json()
            current_dict = {}
            dict_milestone = {}
            milestones = []
            key_mapping = {
                'blNo': 'BL Number',
                'cntrNo': 'Container Number',
                'etd': 'ETD',
                'eta': 'ETA',
                # 'GTOOB':'Gate Out Origin',
                # 'GTIOB':'Gate In Origin',
                'LDGOB': 'ATD',
                'DISIB': 'ATA',
                'GTOIB': 'Container Release',
                'GTIIB': 'Container Return'
            }

            current_dict.update({"Liners": "KMTC"})
            current_dict.update(
                {"From": milstone_data['trackingList'][-1]['plcNm'][:milstone_data['trackingList'][-1]['plcNm'].index(",")]})
            current_dict.update(
                {"To": milstone_data['trackingList'][-1]['podPortNm']})

            for key, label in key_mapping.items():
                if key in ctr_data['cntrList'][c]:
                    if key == "etd" or key == "eta":
                        current_dict[label] = datetime.strptime(
                            ctr_data['cntrList'][c][key][:8], "%Y%m%d").strftime("%Y-%m-%d")
                    else:
                        current_dict[label] = ctr_data['cntrList'][c][key]

            for b, milestone in enumerate(milstone_data['trackingList']):
                milestones.append(
                    milestone["cntrStsCd"] + milestone["cntrMvntCd"])
                milestones.append(milestone["mvntDt"])

            for m, milestone in enumerate(milestones):
                if is_date(milestone):
                    case_milestone = {
                        milestones[m-1]: datetime.strptime(milestone, "%Y%m%d").strftime("%Y-%m-%d")}
                    dict_milestone.update(case_milestone)

            for key, label in key_mapping.items():
                if key in dict_milestone:
                    current_dict[label] = dict_milestone[key]

            print(current_dict)
            hasil_akhir.append(current_dict)
    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)


  0%|          | 0/13 [00:00<?, ?it/s]

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHEKOU', 'BL Number': 'JKT4599082', 'Container Number': 'BEAU4870270', 'ETD': '2023-09-19', 'ETA': '2023-09-26', 'ATD': '2023-09-19', 'ATA': '2023-09-26', 'Container Release': '2023-09-27', 'Container Return': '2023-09-27'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHEKOU', 'BL Number': 'JKT4599082', 'Container Number': 'BMOU6809231', 'ETD': '2023-09-19', 'ETA': '2023-09-26', 'ATD': '2023-09-19', 'ATA': '2023-09-26', 'Container Release': '2023-10-06', 'Container Return': '2023-10-07'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHEKOU', 'BL Number': 'JKT4599082', 'Container Number': 'BMOU6817432', 'ETD': '2023-09-19', 'ETA': '2023-09-26', 'ATD': '2023-09-19', 'ATA': '2023-09-26', 'Container Release': '2023-10-06', 'Container Return': '2023-10-07'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHEKOU', 'BL Number': 'JKT4599082', 'Container Number': 'BMOU6926639', 'ETD': '2023-09-19', 'ETA': '2023-09-26', 'ATD': '2023-09-19', 'ATA': '2023-0

  8%|▊         | 1/13 [00:19<03:53, 19.50s/it]

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHEKOU', 'BL Number': 'JKT4599082', 'Container Number': 'UETU5452622', 'ETD': '2023-09-19', 'ETA': '2023-09-26', 'ATD': '2023-09-19', 'ATA': '2023-09-26', 'Container Release': '2023-10-07', 'Container Return': '2023-10-08'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHANGHAI', 'BL Number': 'JKT4582808', 'Container Number': 'BMOU6208333', 'ETD': '2023-09-24', 'ETA': '2023-10-06', 'ATD': '2023-09-24', 'ATA': '2023-10-06'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHANGHAI', 'BL Number': 'JKT4582808', 'Container Number': 'GESU6841447', 'ETD': '2023-09-24', 'ETA': '2023-10-06', 'ATD': '2023-09-24', 'ATA': '2023-10-06', 'Container Release': '2023-10-08', 'Container Return': '2023-10-08'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHANGHAI', 'BL Number': 'JKT4582808', 'Container Number': 'HJMU1555665', 'ETD': '2023-09-24', 'ETA': '2023-10-06', 'ATD': '2023-09-24', 'ATA': '2023-10-06', 'Container Release': '2023-10-08', 'Container Return': '

 15%|█▌        | 2/13 [00:34<03:06, 16.92s/it]

...

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4587697', 'Container Number': 'SEGU6586346', 'ETD': '2023-09-14', 'ETA': '2023-09-20', 'ATD': '2023-09-14', 'ATA': '2023-09-20'}


 92%|█████████▏| 12/13 [01:12<00:02,  2.86s/it]

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'NORTH MANILA', 'BL Number': 'JKT4601010', 'Container Number': 'FFAU1014167', 'ETD': '2023-09-26', 'ETA': '2023-10-04', 'ATD': '2023-09-26', 'ATA': '2023-10-04'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'XINGANG', 'BL Number': 'JKT4587839', 'Container Number': 'BEAU4732429', 'ETD': '2023-09-15', 'ETA': '2023-09-30', 'ATD': '2023-09-15', 'ATA': '2023-09-30', 'Container Release': '2023-10-05', 'Container Return': '2023-10-06'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'XINGANG', 'BL Number': 'JKT4587839', 'Container Number': 'BEAU4738772', 'ETD': '2023-09-15', 'ETA': '2023-09-30', 'ATD': '2023-09-15', 'ATA': '2023-09-30', 'Container Release': '2023-10-05', 'Container Return': '2023-10-06'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'XINGANG', 'BL Number': 'JKT4587839', 'Container Number': 'BMOU6373174', 'ETD': '2023-09-15', 'ETA': '2023-09-30', 'ATD': '2023-09-15', 'ATA': '2023-09-30', 'Container Release': '2023-10-03', 'Container Return'

100%|██████████| 13/13 [01:15<00:00,  5.80s/it]

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'XINGANG', 'BL Number': 'JKT4587839', 'Container Number': 'TXGU5007140', 'ETD': '2023-09-15', 'ETA': '2023-09-30', 'ATD': '2023-09-15', 'ATA': '2023-09-30', 'Container Release': '2023-10-05', 'Container Return': '2023-10-05'}


In [6]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(hasil_akhir)

Today's Collection Name ===>  all_tracking_Oct-09-2023
Today's Collection Has Been Made
Inserting Many Complete!!


In [7]:
gagal

[]

In [7]:
# exporting to excel file
df = pd.DataFrame(hasil_akhir)
excel_file_path = 'export excel/KMTC.xlsx'
df.to_excel(excel_file_path, index=False)